In [1]:
import torch
import torch.nn as nn

In [54]:
class ChannelPool(nn.Module):

  def forward(self,x):
    return torch.cat([torch.unsqueeze(torch.max(x,1).values,1),torch.unsqueeze(torch.mean(x,1),1)],dim=1)

In [55]:
class SpatialAttention(nn.Module):
    def __init__(self,use_relu=False,kernel_size=7):
        super(SpatialAttention,self).__init__()
        
        self.use_relu=use_relu
        self.ch_pool = ChannelPool()
        self.conv = nn.Conv2d(in_channels=2,out_channels=1,padding="same",kernel_size=kernel_size)
        self.batch_norm = nn.BatchNorm2d(1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self,x):
        input_tensor = torch.clone(x)
        x = self.ch_pool(x)
        x = self.conv(x)

        if self.use_relu:
            x = self.relu(x)
        
        x = self.batch_norm(x)

        attention_mask = self.sigmoid(x)
        
        return input_tensor*attention_mask

In [56]:
class ChannelAttention(nn.Module):
  
      def __init__(self,c):
          super(ChannelAttention, self).__init__()
          self.Avg = nn.AdaptiveAvgPool2d((1,1))
          self.Max = nn.AdaptiveMaxPool2d((1,1))

          self.reduction_ratio = c//2 
          self.fc1 = nn.Linear(c,self.reduction_ratio)
          self.relu = nn.ReLU()
          self.fc2 = nn.Linear(self.reduction_ratio,c)
          self.sigmoid = nn.Sigmoid()

      def forward(self,x):
          input_tensor = torch.clone(x)

          x_avg = self.Avg(x)
          x_avg = torch.flatten(x_avg,start_dim=1)
          x_max = self.Max(x)
          
          
          x_max = torch.flatten(x_max,start_dim=1)
       
          
          
          out1 = self.fc1(x_avg)
         
          out1 = self.relu(out1)
          out1 = self.fc2(out1)


          out2 = self.fc1(x_max)
          out2 = self.relu(out2)
          out2 = self.fc2(out2)

          out = out1+out2

          out = self.sigmoid(out)
          out = torch.unsqueeze(torch.unsqueeze(out,-1),-1)

          return input_tensor*out








In [57]:
class CBAM(nn.Module):
    def __init__(self,in_channels):
     super(CBAM, self).__init__()
     self.in_channels = in_channels
     self.ch_attn = ChannelAttention(self.in_channels)
     self.sp_attn = SpatialAttention()

    def forward(self,x):
       
        out = self.ch_attn(x)
        out = self.sp_attn(out)

        return out


